<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/Agents/pydantic_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install 'pydantic-ai-slim[openai]'

# Step 1: Basic model connector


### Imports and env variables

First we import libs and create env variable for `OpenAIChatModel` to be able to get `OPENROUTER_API_KEY` from env variables

In [6]:
import os

from google.colab import userdata
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openrouter import OpenRouterProvider


OPENROUTER_API_KEY = userdata.get('openrouter')
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

### Initiating model and agent


We create an agent output scheme

In [4]:
class Result(BaseModel):
    answer: str
    confidence: float

> Write system prompt

> create model

> create agent

In [12]:
sys_prompt = "You're a study assistant agent. Your answer should be brief and structured. Avoid emoji and slang."

model = OpenAIChatModel("deepseek/deepseek-chat-v3.1:free",
                        provider=OpenRouterProvider())


agent = Agent[None, Result](model=model,
                            system_prompt=sys_prompt)

### Handling `agent.run` and memory

Simple async run function with an agent decorator. In Pydantic AI the context "lives" while the `agent.run` is being executed (unlike usual LLM request `model.complete` or `model.generate` when it doesn't have access to it's own history/tool use), however between requests there is no memory.

The agent has access to `ctx.deps` - like ram and `ctx.memory_history` - all prompts to the model in current run, after `agent.run` return result the session is closed and the memory is erased.

In [13]:
@agent.run
async def run(ctx: RunContext[None], question: str) -> Result:
    draft = await ctx.llm.complete(question)
    return Result(answer=draft.data, confidence=0.5)

/tmp/ipython-input-1231992350.py:2: RuntimeWarning: coroutine 'AbstractAgent.run' was never awaited
  async def run(ctx: RunContext[None], question: str) -> Result:


### Try it

In [14]:
result = await agent.run("Привет, кто ты?")
print(result)


AgentRunResult(output='Я — ваш помощник-ассистент, предназначенный для ответов на вопросы и помощи в учёбе. Чем могу помочь?')


## Step 2: add tools

### More imports

In [16]:
from typing import Dict, Any, Optional, List
from pydantic import Field
import ast, operator as op

### Create schema for dependencies - data accessible to the Agent during agent.run
We create pydantic model so it would follow typisation, and get validated
When `agent.run` is executed `Deps` gets instantiated and is accessible via `ctx.deps`

> `scratch` - is just a field for the agent to dump info to: numbers, dates, notes, any important data
> `tool_call` - logging of all tool calls: tool, args, results.

*Class attributes above could be any other names*

In [17]:
class Deps(BaseModel):
    scratch: Dict[str, Any] = Field(default_factory=dict)
    tool_calls: List[Dict[str, Any]] = Field(default_factory=list)

### Create a new system prompt and agent

In [18]:
sys_prompt = """You are a calculation agent.
- Always use the provided tools (calculator, unit converter, evaluator) to verify answers.
- First, generate a draft answer for the user’s question.
- Then compare the draft to the tool’s result.
- If they match, return the draft as final.
- If they differ, correct the answer using the tool result and explain briefly.
Keep answers short and precise."""


agent = Agent[Deps, Result](model=model,
                            system_prompt=sys_prompt)

### Tool 1: calculator

In [23]:
class CalcInput(BaseModel):
    expression: str

class CalcOutput(BaseModel):
    value: float

In [24]:
_ALLOWED_OPS = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
    ast.Div: op.truediv, ast.Pow: op.pow, ast.Mod: op.mod,
    ast.USub: op.neg, ast.UAdd: op.pos,
}

def _safe_eval(node):
    if isinstance(node, ast.Num):  # py<=3.7
        return node.n
    if isinstance(node, ast.Constant):  # py 3.8+
        if isinstance(node.value, (int, float)):
            return node.value
        raise ValueError("constants other than numbers are not allowed")
    if isinstance(node, ast.UnaryOp) and type(node.op) in _ALLOWED_OPS:
        return _ALLOWED_OPS[type(node.op)](_safe_eval(node.operand))
    if isinstance(node, ast.BinOp) and type(node.op) in _ALLOWED_OPS:
        return _ALLOWED_OPS[type(node.op)](_safe_eval(node.left), _safe_eval(node.right))
    if isinstance(node, ast.Expr):
        return _safe_eval(node.value)
    raise ValueError("unsupported expression")


In [ ]:
@agent.tool
def calc_expr(ctx: RunContext[Deps], data: CalcInput) -> CalcOutput:
    # + - * / ** %
    tree = ast.parse(data.expression, mode="eval")
    val = float(_safe_eval(tree.body))
    ctx.deps.tool_calls.append({"tool": "calc_expr", "expr": data.expression, "value": val})
    return CalcOutput(value=val)